In [55]:
USER = 'thomas'

In [56]:
import numpy as np
import pandas as pd
import os
import sys

In [57]:
sys.path.insert(0, os.path.join(os.path.abspath('..'), 'dataset'))

In [58]:
from load_movies import load_movies

In [59]:
allmovies = load_movies()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [60]:
bymovie = allmovies[['movieId', 'rating', 'vote_count']].groupby('movieId', sort=False).mean()

In [61]:
over1000votes = bymovie[bymovie['vote_count'] >= 1000]

In [62]:
def restrict_ratings(ratings, low, high):
    '''Return `ratings` with just ratings in [low, high)'''
    return ratings[(ratings['rating'] <= high) & (ratings['rating'] > low)]

In [63]:
def collect_movies(ratings, unique_ranges, keep_top_n=50):
    movies = pd.DataFrame(columns=ratings.columns, index=ratings.index)
    for low_i, high_i in unique_ranges:
        best_in_this_range = restrict_ratings(ratings, low_i, high_i).sort_values('vote_count').iloc[-keep_top_n:]
        movies = pd.concat([movies, best_in_this_range])
    return movies.dropna()

In [64]:
UNIQUE_RANGES = ((0, 0.5), (0.5, 1), (1, 1.5), (1.5, 2), (2, 2.5), (2.5, 3), (3, 3.5), (3.5, 5))

In [65]:
def convert_to_save_format(collected, allmovies):
    return pd.merge(collected.drop(['rating', 'vote_count'], axis=1).reset_index(),
                    allmovies[['imdb_id', 'title', 'movieId']].drop_duplicates(),
                    on='movieId').drop('movieId', axis=1)

In [66]:
def save_collected(collected, allmovies, savepath):
    convert_to_save_format(collected, allmovies).to_csv(savepath, index=False)

In [67]:
save_collected(collect_movies(over1000votes, UNIQUE_RANGES), allmovies, f'movies_for_{USER}_to_rate.csv')